In [ ]:
import MySQLdb
from pandas.io import sql
import pandas as pd
import numpy as np
conn = MySQLdb.connect(host="localhost", user="root", passwd="MYSQLROOTPASSWORD", db="openmrs") 
openmrscursor = conn.cursor() 

In [ ]:
mimicconn = MySQLdb.connect(host="localhost", user="root", passwd="MYSQLROOTPASSWORD", db="mimic4") 
mimiccursor = mimicconn.cursor()
df_icustays_mimic = pd.read_sql('select * from icustays', con = mimicconn) 
df_icustays_mimic

In [ ]:
df_icustays_mimic['subject_id'].nunique()

In [ ]:
Unique_fisrtcare_unit = pd.read_sql('SELECT distinct first_careunit FROM mimic4.icustays', con = mimicconn) 
Unique_fisrtcare_unit

In [ ]:
df_location = pd.read_sql('select * from `location`', con = conn) 
df_location = "INSERT INTO location(location_id, name) VALUES (2, 'Trauma SICU (TSICU)'), (3, 'Neuro Stepdown'), (4, 'Coronary Care Unit (CCU)'), (5, 'Neuro Intermediate'),
                (6, 'Medical Intensive Care Unit (MICU)'), (7, 'Cardiac Vascular Intensive Care Unit (CVICU)'),
                (8, 'Surgical Intensive Care Unit (SICU)'), (9, 'Neuro Surgical Intensive Care Unit (Neuro SICU)'),
                (10, 'Medical/Surgical Intensive Care Unit (MICU/SICU)')"
openmrscursor.execute(df_location)
conn.commit()

In [ ]:
Unique_Lastcare_unit = pd.read_sql('SELECT distinct last_careunit FROM mimic4.icustays', con = mimicconn) 
Unique_Lastcare_unit

In [ ]:
df_encounter_type = pd.read_sql('select * from `encounter_type`', con = conn) 
df_encounter_type = "INSERT INTO encounter_type(encounter_type_id, name, description, creator, date_created, uuid) VALUES (2, 'mimic-icu-encounter', 'Data that is imported from mimic-iv', 1, '2022-04-13', uuid())"
openmrscursor.execute(df_encounter_type)
conn.commit()
df_encounter_type

In [ ]:
df_location = pd.read_sql('select * from `location`', con = conn) 
df_location

In [ ]:
df_icustays_mimic

In [ ]:
dict_ = {'Trauma SICU (TSICU)' : 2, 'Neuro Stepdown' : 3, 'Coronary Care Unit (CCU)' : 4, 
         'Neuro Intermediate' : 5, 'Medical Intensive Care Unit (MICU)':6, 'Cardiac Vascular Intensive Care Unit (CVICU)': 7,
        'Surgical Intensive Care Unit (SICU)':8, 'Neuro Surgical Intensive Care Unit (Neuro SICU)': 9, 
        'Medical/Surgical Intensive Care Unit (MICU/SICU)': 10}
df_icustays_mimic=df_icustays_mimic.replace({"first_careunit": dict_})
df_icustays_mimic=df_icustays_mimic.replace({"last_careunit": dict_})

In [ ]:
df_icustays_mimic

In [ ]:
df_icustays_mimic_rename = df_icustays_mimic.rename({'subject_id': 'patient_id', 'hadm_id':'visit_id','stay_id':'encounter_id', 'intime':'encounter_datetime', 'first_careunit':'location_id' }, axis='columns')
df_icustays_mimic_rename['date_created'] = '2022-01-01'
df_icustays_mimic_rename['encounter_type'] = 2

In [ ]:
import uuid
df_icustays_mimic_rename['uuid'] = [uuid.uuid4() for x in range(len(df_icustays_mimic_rename))]

In [ ]:
df_icustays_mimic_final = df_icustays_mimic_rename.drop(columns =['last_careunit', 'outtime', 'los' ])
df_icustays_mimic_final 

In [ ]:
# creating column list for insertion for encounter table
cols = "`,`".join([str(i) for i in df_icustays_mimic_final.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in df_icustays_mimic_final.iterrows():
    sql1 = "SET FOREIGN_KEY_CHECKS=0"
    sql = "INSERT INTO `encounter` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    sql2 = "SET FOREIGN_KEY_CHECKS=1"
    try:
        openmrscursor.execute(sql1)
        openmrscursor.execute(sql, tuple(row))
        openmrscursor.execute(sql2)
        
    except MySQLdb.OperationalError:
        continue

    # the connection is not autocommitted by default, so we must commit to save our changes
    conn.commit()

encounter_creator = 'UPDATE encounter SET creator=1'
encounter_form_id = 'UPDATE encounter SET form_id = NULL' 

openmrscursor.execute(encounter_creator)
openmrscursor.execute(encounter_form_id)

conn.commit() 

In [ ]:
df_encounter = pd.read_sql('select * from `encounter`', con = conn) 
df_encounter